In [1]:
import torch 

In [2]:
from IPython.display import Image #this is to render predictions

In [3]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [3]:
%cd yolov5

C:\Users\parth\Desktop\yolov5 pytorch\yolov5


In [4]:
!pip install -r requirements.txt

In [5]:
from tqdm import tqdm
import IProgress
from ipywidgets import IntProgress

In [18]:
#torch.cuda.get_arch_list() 

In [6]:
print('All set. Using PyTorch version %s with %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

All set. Using PyTorch version 1.13.1+cpu with CPU


# Training the yolov5 Model

 Once all the files are cloned, you’ll need to move the dataset’s dataset.yaml file to the yolov5/data directory. 
 This file contains information required by YOLO to train the model on the custom data. 

In [9]:
!python train.py --img 415 --batch 3 --epochs 60 --data dataset.yaml --weights yolov5s.pt --cache


At the end of the training, two files should be saved in yolov5/runs/train/exp/weights: last.pt and best.pt. We’ll use best.pt.

If you want to explore the metrics recorded during training, I suggest you use TensorBoard, a very interactive exploration tool:

In [10]:
%load_ext tensorboard
%tensorboard --logdir runs

# Testing the Model on Google Colab
Let’s explore now how confident our model is. We can plot a validation batch obtained during training and inspect the confidence score of each label:

In [13]:
Image(filename='runs/train/exp2/val_batch0_labels.jpg', width=1000)

NameError: name 'Image' is not defined

# Testing the Model:

In [14]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 416 --conf 0.25 --source runs/train/exp2/test_images

detect: weights=['runs/train/exp2/weights/best.pt'], source=runs/train/exp2/test_images, data=data\coco128.yaml, imgsz=[416, 416], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to 'C:\Users\parth\Desktop\yolov5': No such file or directory
YOLOv5  2023-7-5 Python-3.7.16 torch-1.13.1+cpu CPU

Traceback (most recent call last):
  File "detect.py", line 261, in <module>
    main(opt)
  File "detect.py", line 256, in main
    run(**vars(opt))
  File "C:\Users\parth\anaconda3\envs\parthenv\lib\site-packages\torch\autograd\grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 98, in run
    model = DetectMultiBackend(weight

# Yolov5 model is trained and tested. 

# Now next step is to convert it into a tflite model, so that we can use it on android phone


### you have best.pt and last.pt inside runs\train\exp2\weights

In [ ]:
# Convert weights to fp16 TFLite model

!python export.py --weights runs/train/exp2/weights/best.pt --include tflite --img 416

In [ ]:
#After running the above command, you will have best-fp16.tflite in runs\train\exp2\weights folder

# Lets run the created tflite model.

!python detect.py --weights runs\train\exp2\weights\best-fp16.tflite --img 416 --conf 0.25 --source runs/train/exp2/test_images

# Our model is working fine and results are stored in runs\detect\exp3

In [ ]:
#Convert weights to int8 TFLite model

#!python export.py --weights runs/train/exp2/weights/best.pt --include tflite --int8 --img 416 --data data/coco128.yaml

## 3. Get the android folder from here
https://github.com/AarohiSingla/TFLite-Object-Detection-Android-App-Tutorial-Using-YOLOv5

### Now Put TFLite models in assets folder of Android project, and change

And then open android/app/src/main/java/org/tensorflow/lite/examples/detection/tflite/DetectorFactory.java

Do the following changes:

Change the inputSize of image as per your dataset.

Change labelFilename according to the classes of the model

    
Then run the program in Android Studio.